In [47]:
import numpy as np
import pandas as pd
from copy import copy
from sklearn.preprocessing import LabelEncoder
import os
for dirname, _, filenames in os.walk('kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import warnings
warnings.filterwarnings('ignore')

In [48]:
data = pd.read_csv('DATA_STUDENTS.csv', header=None)

new_header = data.iloc[458]

data = data.rename(columns=new_header)

data = data.drop(459).reset_index(drop=True)

print(f"\nТипы данных в столбцах: \n{data.dtypes}")

#Данные, исходя из вывода имеют тип object. Меняем его на нормальный.
numeric_cols = ['Age_Y', 'Weight_Kg', 'Height_cm', 'BMI', 'Speed_m/s', 'Jump _distance_cm']
data[numeric_cols] = data[numeric_cols].apply(pd.to_numeric, errors='coerce')

print(f"\nТипы данных в столбцах: \n{data.dtypes}")



Типы данных в столбцах: 
ID                   object
Age_Y                object
Gender               object
Weight_Kg            object
Height_cm            object
BMI                  object
Speed_m/s            object
Jump _distance_cm    object
Region               object
dtype: object

Типы данных в столбцах: 
ID                    object
Age_Y                float64
Gender                object
Weight_Kg            float64
Height_cm            float64
BMI                  float64
Speed_m/s            float64
Jump _distance_cm    float64
Region                object
dtype: object


In [49]:
data

,ID,Age_Y,Gender,Weight_Kg,Height_cm,BMI,Speed_m/s,Jump _distance_cm,Region
0,371,12.0,male,30.0,138.0,16.0,6.17,321.0,Bogota
1,636,13.0,male,31.0,141.0,16.0,4.81,280.0,Bogota
2,666,13.0,male,31.0,142.0,15.0,5.43,310.0,Bogota
3,1548,13.0,male,31.0,142.0,15.0,4.70,320.0,Bogota
4,358,13.0,male,31.0,145.0,15.0,5.62,245.0,Bogota
...,...,...,...,...,...,...,...,...,...
1852,106,12.0,male,42.5,160.0,17.0,5.41,360.0,Bogota
1853,1693,11.0,male,42.5,160.0,17.0,5.40,360.0,Cairo
1854,1611,15.0,female,50.3,162.0,19.0,5.40,243.0,Riga
1855,310,14.0,female,58.5,155.0,24.0,5.33,270.0,Riga


In [50]:
# Размерность данных
print(f"Размер данных: {data.shape}")

# Пропуски
print("\nКоличество пропущенных значений:")
print(data.isnull().sum())

Размер данных: (1857, 9)

Количество пропущенных значений:
ID                   0
Age_Y                1
Gender               0
Weight_Kg            1
Height_cm            1
BMI                  1
Speed_m/s            8
Jump _distance_cm    1
Region               0
dtype: int64


Удаляем пропуски


In [51]:
# Удаляем строки с пропущенными значениями
data = data.dropna()

# Сбрасываем индексы после удаления строк
data = data.reset_index(drop=True)

# Проверяем количество пропусков после очистки
print("\nКоличество пропущенных значений после очистки:")
print(data.isnull().sum())


Количество пропущенных значений после очистки:
ID                   0
Age_Y                0
Gender               0
Weight_Kg            0
Height_cm            0
BMI                  0
Speed_m/s            0
Jump _distance_cm    0
Region               0
dtype: int64


Межквартильный размах определяется через квантили - разница между 75-м и 25-м процентилям.

In [52]:
#Да, функция для одного применения, но я предпочту сделать так, потому что можно будет сунуть в библиотеку.
def remove_outliers_iqr(df, columns):
    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    return df.reset_index(drop=True)

# фильтрация
data_clean = remove_outliers_iqr(data, numeric_cols)

# Проверка размерности
print("До удаления выбросов:", data.shape)
print("После удаления выбросов:", data_clean.shape)

До удаления выбросов: (1849, 9)
После удаления выбросов: (1689, 9)


Борьба с шумами

In [53]:
def smooth_columns(df, columns, window=3):
    df_copy = df.copy()
    for col in columns:
        df_copy[col] = df_copy[col].rolling(window=window, center=True, min_periods=1).mean()
    return df_copy

data_smoothed = smooth_columns(data_clean, numeric_cols)

Удаляем дубликаты (их не оказалось)

In [54]:
data_nodup = data_clean.drop_duplicates().reset_index(drop=True)

print(data_smoothed.shape)

(1689, 9)


Этот код, чтобы отмести котегориальные поля я просто взял со своего старого проекта:

https://github.com/delilit/Spotify

In [55]:
object_columns = data_nodup.select_dtypes(include=['object']).columns
labels = {}

for col in object_columns:
    le = LabelEncoder()
    data_nodup[col] = le.fit_transform(data_nodup[col])
    labels[col] = le

data_nodup.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1688 entries, 0 to 1687
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ID                 1688 non-null   int64  
 1   Age_Y              1688 non-null   float64
 2   Gender             1688 non-null   int64  
 3   Weight_Kg          1688 non-null   float64
 4   Height_cm          1688 non-null   float64
 5   BMI                1688 non-null   float64
 6   Speed_m/s          1688 non-null   float64
 7   Jump _distance_cm  1688 non-null   float64
 8   Region             1688 non-null   int64  
dtypes: float64(6), int64(3)
memory usage: 118.8 KB
